# class-ratio-maintained

In [1]:
# these warnings are fine. you can ignore them.
import random, math

import sys
sys.path.append('../')
from util import *

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Imports finished.")

/home/wcorcoran/anaconda3/envs/mesp/lib/python3.8/site-packages/torch_geometric/typing.py:47: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: /lib64/libm.so.6: version `GLIBC_2.27' not found (required by /home/wcorcoran/anaconda3/envs/mesp/lib/python3.8/site-packages/libpyg.so)
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
/home/wcorcoran/anaconda3/envs/mesp/lib/python3.8/site-packages/torch_geometric/typing.py:101: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /lib64/libm.so.6: version `GLIBC_2.27' not found (required by /home/wcorcoran/anaconda3/envs/mesp/lib/python3.8/site-packages/libpyg.so)
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "


KeyboardInterrupt: 

# Setting Up Dataset/Model/Ground Truth

In [ ]:
dataset = Dataset(root='/tmp/Cora', name='Cora', device=device)
data, in_feats, h_feats, num_classes = dataset.get_data()

model = get_model(in_feats, h_feats, num_classes, "cora")
ground_truth = get_ground_truth(model, data)
print(ground_truth)

# Experiments
**Note:** The ground truth here is $0.749$

Experiment ideas:
- 

In [ ]:
homophilic_set = {label: [] for label in set(data.y.tolist())}

# test_indices = torch.nonzero(data.test_mask, as_tuple=False).squeeze()
G, x, y, train_mask, test_mask = convert_to_networkx(dataset.get_data()[0])
for i in G.nodes():
    homophilic_set[data.y[i].item()].append(i)

# print("This is the dictionary containing each class and its respective vertices:\n\t", homophilic_set)

### Experiment 1
First, run connected components on the graph.
Let $C$ indicate the classes. For each node, $u$, in a connect component, $z$, calculate the class-connectivity rate, $r(u, c)$, where $c \in \text{C}$:
$$r(u, c) = \sum_{v \in N(u)}\frac{[[C(v) = c]]}{|N(u)|}$$
(i.e. the number of edges to different classes divided by total size of the neighborhood).

Then, add edges to this node in proportion (or up to) to the previous rates (for example, if there are 3 classes and a node has edges like {class: #}: {1: 2}, {2: 1}, {3:0}, then, if you *doubled* the number of edges, you would have {1: 4}, {2: 2}, {3: 0}.

In [9]:
# data = dataset.get_data()[0]
# modified_graph = data

# init_edges = len(modified_graph.edge_index[1])

# G, x, y, train_mask, test_mask = convert_to_networkx(modified_graph)

# cc = sorted(nx.strongly_connected_components(G), key=len, reverse=True)

# # store before alteration dict
# # after alteration dict
# n_dict = {i: ({}, {}) for i in range(0, G.number_of_nodes())}

# def getClassConnectivity(n, cc, y, num_classes):
#     dict = {i: 0 for i in range(0, num_classes)}
#     for neighbor in G.adj[n]:
#         dict[y[neighbor].item()] += 1
#     return dict

# def increaseRCC(G, n, m_cc, n_dict, val, y):
#     # print(n)
#     # check floors - so check if altered (second dict != floor(first dict * (1 + val))
#     # first, iterate over cc to check that it's possible
#     # check that there exists floor(fd * (1 + val)) - fd nodes of each class which are unconnected
#     # otherwise, exit
#     for i in range(0, len(n_dict[n][0])):
#         print("n: ", n, "i: ", i, "len cc:", len(m_cc))
#         num = 0
#         for j in m_cc:
#             if n != j and y[j].item() == i and not G.has_edge(n, j):
#                 num += 1
#         if not num >= math.floor(n_dict[n][0][i] * (1 + val)) - n_dict[n][1][i]:
#             return

#     # # then, iterate over all other nodes
#     # # check, class, if satiated, don't add
#     # # if not satiated, go until satiated
#     # # make sure to add to both sides u and v
#     # for i in range(0, len(n_dict[n][0])):
#     #     goal = math.floor(n_dict[n][0][i] * (1 + val))
#     #     t_cc = m_cc 
#     #     random.shuffle(list(t_cc))
#     #     for j in t_cc:
#     #         if n != j and y[j].item() == i and not G.has_edge(n, j) and n_dict[n][1][i] < goal and n_dict[j][1][y[n].item()] < math.floor(n_dict[j][0][y[n].item()] * (1 + val)):
#     #             # print("added edge, n: ", n, "j: ", j)
#     #             add_edge(G, n, j, undirected=True)
#     #             n_dict[n][1][i] += 1
#     #             n_dict[j][1][y[n].item()] += 1
                
                            
# for i in cc:
#     for n in i:
#         n_dict[n][0].update(getClassConnectivity(n, cc, y, num_classes))
#         n_dict[n][1].update(getClassConnectivity(n, cc, y, num_classes))

# c = 1
# num = 0
# for i in cc:
#     for n in i:
#         # print(n)
#         # print(i)
#         increaseRCC(G, n, i, n_dict, c, y)
#         num += 1
#         if (num >= 50):
#             break

# print(n_dict)

# modified_graph = convert_to_pyg(G, x, y, train_mask, test_mask)
# final_edges = len(modified_graph.edge_index[1])

# output_accuracy_change(ground_truth, test_model(model, modified_graph)) 
# number_added_edges(init_edges, final_edges, is_undirected=True)

# for above, copy this? but just recalculate the homophilic set each time?
# for each cc?
# must check that edges can be added
def getHomophilicSet(G, cc, y):
    homophilic_set = {label: [] for label in set(y.tolist())}

    for i in cc:
        homophilic_set[y[i].item()].append(i)

    return homophilic_set

for i in range(1, 21):
    constant = i / 20
    
    data = dataset.get_data()[0]
    modified_graph = data
    
    init_edges = len(modified_graph.edge_index[1])
    
    G, x, y, train_mask, test_mask = convert_to_networkx(modified_graph)
    cc = sorted(nx.strongly_connected_components(G), key=len, reverse=True)
    
    # store before alteration dict
    # after alteration dict
    n_dict = {i: ({}, {}) for i in range(0, G.number_of_nodes())}
    
    for n in G.nodes():
            n_dict[n][0].update(getClassConnectivity(G, n, y, num_classes))
            n_dict[n][1].update(getClassConnectivity(G, n, y, num_classes))
    
    nodes = G.nodes()

    for i in cc:
        hset = getHomophilicSet(G, i, y)
        for n in i:
            for c in range(0, len(hset)):
                s = hset[c].copy()
                while len(s) > 0 and not isSatisfied(n, c, n_dict, constant):
                    temp = random.choice(s)
                    s.remove(temp)
                    if n != temp and not isSatisfied(temp, y[n].item(), n_dict, constant):
                        add_edge(G, n, temp, undirected=True)
                        n_dict[n][1][c] += 1
                        n_dict[temp][1][y[n].item()] += 1
        
    modified_graph = convert_to_pyg(G, x, y, train_mask, test_mask)
    final_edges = len(modified_graph.edge_index[1])
    
    output_accuracy_change(ground_truth, test_model(model, modified_graph)) 
    number_added_edges(init_edges, final_edges, is_undirected=True)
    print("For c value:", constant)


----
The accuracy has changed by 0.0010
Change in edges:  6.0  | Percentage change: 0.11%
For c value: 0.05

----
The accuracy has changed by 0.0020
Change in edges:  41.0  | Percentage change: 0.78%
For c value: 0.1

----
The accuracy has changed by 0.0040
Change in edges:  118.0  | Percentage change: 2.24%
For c value: 0.15

----
The accuracy has changed by 0.0100
Change in edges:  291.0  | Percentage change: 5.51%
For c value: 0.2

----
The accuracy has changed by 0.0140
Change in edges:  525.0  | Percentage change: 9.95%
For c value: 0.25

----
The accuracy has changed by 0.0140
Change in edges:  569.0  | Percentage change: 10.78%
For c value: 0.3

----
The accuracy has changed by 0.0330
Change in edges:  920.0  | Percentage change: 17.43%
For c value: 0.35

----
The accuracy has changed by 0.0360
Change in edges:  1063.0  | Percentage change: 20.14%
For c value: 0.4

----
The accuracy has changed by 0.0330
Change in edges:  1095.0  | Percentage change: 20.75%
For c value: 0.45

-

The above was too slow.
This next one will do a similar experiment but without considering connect components.

In [5]:
for i in range(1, 21):
    constant = i / 20
    
    data = dataset.get_data()[0]
    modified_graph = data
    
    init_edges = len(modified_graph.edge_index[1])
    
    G, x, y, train_mask, test_mask = convert_to_networkx(modified_graph)
    
    # store before alteration dict
    # after alteration dict
    n_dict = {i: ({}, {}) for i in range(0, G.number_of_nodes())}
    
    for n in G.nodes():
            n_dict[n][0].update(getClassConnectivity(G, n, y, num_classes))
            n_dict[n][1].update(getClassConnectivity(G, n, y, num_classes))
    
    nodes = G.nodes()
    for n in nodes:
        for c in range(0, len(homophilic_set)):
            s = homophilic_set[c].copy()
            while len(s) > 0 and not isSatisfied(n, c, n_dict, constant):
                temp = random.choice(s)
                s.remove(temp)
                if n != temp and not isSatisfied(temp, y[n].item(), n_dict, constant):
                    add_edge(G, n, temp, undirected=True)
                    n_dict[n][1][c] += 1
                    n_dict[temp][1][y[n].item()] += 1
        
    modified_graph = convert_to_pyg(G, x, y, train_mask, test_mask)
    final_edges = len(modified_graph.edge_index[1])
    
    output_accuracy_change(ground_truth, test_model(model, modified_graph)) 
    number_added_edges(init_edges, final_edges, is_undirected=True)
    print("For c value:", constant)


----
The accuracy has changed by 0.0010
Change in edges:  5.0  | Percentage change: 0.09%
For c value: 0.05

----
The accuracy has changed by 0.0020
Change in edges:  43.0  | Percentage change: 0.81%
For c value: 0.1

----
The accuracy has changed by 0.0080
Change in edges:  114.0  | Percentage change: 2.16%
For c value: 0.15

----
The accuracy has changed by 0.0080
Change in edges:  299.0  | Percentage change: 5.67%
For c value: 0.2

----
The accuracy has changed by 0.0120
Change in edges:  527.0  | Percentage change: 9.98%
For c value: 0.25

----
The accuracy has changed by 0.0170
Change in edges:  581.0  | Percentage change: 11.01%
For c value: 0.3

----
The accuracy has changed by 0.0280
Change in edges:  941.0  | Percentage change: 17.83%
For c value: 0.35

----
The accuracy has changed by 0.0430
Change in edges:  1084.0  | Percentage change: 20.54%
For c value: 0.4

----
The accuracy has changed by 0.0330
Change in edges:  1155.0  | Percentage change: 21.88%
For c value: 0.45

-